# Genetic algorithm-based heuristic for feature selection in credit risk assessment

First we import libraries ,we will be needed through this notebook : 

In [9]:
import pandas as pd
import numpy as np
from random import random
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import adam

## Configes

In [2]:
config = { 
    'maxFeatureNum': 12,
    'minFeatureNum': 5,
    'populationSize': 50,
    'initProb': .5,
    'crossoverProb': .9
}

## Implementation of Neuralnetwork architecture

In [5]:
def simple_train(X,Y,hiddenNum=40,trainCycles=450,lr=.1,m=.5) :
    """Train one layer feedforward neural network
    Args :
       X : training data
       Y : training label
       hiddenNum : number of hidden units of hidden layer
       trainCycles : number of training cycles
       lr : learning rate of nueral network
       m: momentum of neural network
    Returns :
       'float' accuracy
    """
    model = Sequential()
    model.add(Dense(hiddenNum, input_dim=X.shape[1], kernel_initializer='normal', activation='selu'))
#     model.add(Dense(hiddenNum, kernel_initializer='normal', activation='selu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
#     sgd = SGD(lr=lr, momentum=m)
    ADAM = adam(lr=lr)
    # loss could be "mse" too
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=ADAM)
    model.fit(X,Y,epochs=trainCycles)

In [6]:
x_train = pd.read_excel('GermanCreditInput.xls')
y_train = pd.read_excel('GermanCreditOutputClass1columnknn.xls')
simple_train(np.array(x_train),np.array(y_train).reshape(y_train.shape[0]),30,1000,.0001,.3)

Epoch 1/1000
999/999 [==============================] - 3s 3ms/step - loss: 1.3196 - acc: 0.4925
Epoch 2/1000
999/999 [==============================] - 0s 144us/step - loss: 0.6795 - acc: 0.6987
Epoch 3/1000
999/999 [==============================] - 0s 134us/step - loss: 0.6393 - acc: 0.7007
Epoch 4/1000
999/999 [==============================] - 0s 141us/step - loss: 0.6414 - acc: 0.7017
Epoch 5/1000
999/999 [==============================] - 0s 132us/step - loss: 0.6302 - acc: 0.7057
Epoch 6/1000
999/999 [==============================] - 0s 155us/step - loss: 0.6182 - acc: 0.7007
Epoch 7/1000
999/999 [==============================] - 0s 136us/step - loss: 0.6126 - acc: 0.6997
Epoch 8/1000
999/999 [==============================] - 0s 137us/step - loss: 0.6042 - acc: 0.7057
Epoch 9/1000
999/999 [==============================] - 0s 146us/step - loss: 0.6075 - acc: 0.6997
Epoch 10/1000
999/999 [==============================] - 0s 146us/step - loss: 0.6064 - acc: 0.6967
Epoch 11/10

999/999 [==============================] - 0s 147us/step - loss: 0.5382 - acc: 0.7287
Epoch 83/1000
999/999 [==============================] - 0s 144us/step - loss: 0.5317 - acc: 0.7327
Epoch 84/1000
999/999 [==============================] - 0s 156us/step - loss: 0.5381 - acc: 0.7247
Epoch 85/1000
999/999 [==============================] - 0s 159us/step - loss: 0.5381 - acc: 0.7267
Epoch 86/1000
999/999 [==============================] - 0s 140us/step - loss: 0.5392 - acc: 0.7197
Epoch 87/1000
999/999 [==============================] - 0s 150us/step - loss: 0.5319 - acc: 0.7267
Epoch 88/1000
999/999 [==============================] - 0s 472us/step - loss: 0.5338 - acc: 0.7247
Epoch 89/1000
999/999 [==============================] - 0s 132us/step - loss: 0.5309 - acc: 0.7297
Epoch 90/1000
999/999 [==============================] - 0s 148us/step - loss: 0.5383 - acc: 0.7197
Epoch 91/1000
999/999 [==============================] - 0s 150us/step - loss: 0.5382 - acc: 0.7227
Epoch 92/1000


999/999 [==============================] - 0s 147us/step - loss: 0.4914 - acc: 0.7538
Epoch 164/1000
999/999 [==============================] - 0s 145us/step - loss: 0.4960 - acc: 0.7477
Epoch 165/1000
999/999 [==============================] - 0s 146us/step - loss: 0.5262 - acc: 0.7367
Epoch 166/1000
999/999 [==============================] - 0s 149us/step - loss: 0.5002 - acc: 0.7457
Epoch 167/1000
999/999 [==============================] - 0s 145us/step - loss: 0.4903 - acc: 0.7608
Epoch 168/1000
999/999 [==============================] - 0s 157us/step - loss: 0.5010 - acc: 0.7437
Epoch 169/1000
999/999 [==============================] - 0s 140us/step - loss: 0.4966 - acc: 0.7477
Epoch 170/1000
999/999 [==============================] - 0s 454us/step - loss: 0.5065 - acc: 0.7417
Epoch 171/1000
999/999 [==============================] - 0s 164us/step - loss: 0.5017 - acc: 0.7508
Epoch 172/1000
999/999 [==============================] - 0s 143us/step - loss: 0.4926 - acc: 0.7518
Epoch

999/999 [==============================] - 0s 178us/step - loss: 0.5242 - acc: 0.7568
Epoch 245/1000
999/999 [==============================] - 0s 161us/step - loss: 0.4892 - acc: 0.7658
Epoch 246/1000
999/999 [==============================] - 0s 160us/step - loss: 0.4850 - acc: 0.7728
Epoch 247/1000
999/999 [==============================] - 0s 162us/step - loss: 0.4757 - acc: 0.7748
Epoch 248/1000
999/999 [==============================] - 0s 165us/step - loss: 0.4717 - acc: 0.7818
Epoch 249/1000
999/999 [==============================] - 0s 163us/step - loss: 0.4940 - acc: 0.7598
Epoch 250/1000
999/999 [==============================] - 0s 142us/step - loss: 0.4791 - acc: 0.7748
Epoch 251/1000
999/999 [==============================] - 0s 150us/step - loss: 0.4757 - acc: 0.7788
Epoch 252/1000
999/999 [==============================] - 0s 145us/step - loss: 0.4781 - acc: 0.7818
Epoch 253/1000
999/999 [==============================] - 0s 148us/step - loss: 0.4746 - acc: 0.7718
Epoch

999/999 [==============================] - 0s 188us/step - loss: 0.4917 - acc: 0.7778
Epoch 326/1000
999/999 [==============================] - 0s 175us/step - loss: 0.4990 - acc: 0.7708
Epoch 327/1000
999/999 [==============================] - 0s 159us/step - loss: 0.4654 - acc: 0.7768
Epoch 328/1000
999/999 [==============================] - 0s 163us/step - loss: 0.4710 - acc: 0.7778
Epoch 329/1000
999/999 [==============================] - 0s 159us/step - loss: 0.4900 - acc: 0.7748
Epoch 330/1000
999/999 [==============================] - 0s 154us/step - loss: 0.4727 - acc: 0.7718
Epoch 331/1000
999/999 [==============================] - 0s 157us/step - loss: 0.4735 - acc: 0.7748
Epoch 332/1000
999/999 [==============================] - 0s 158us/step - loss: 0.4752 - acc: 0.7778
Epoch 333/1000
999/999 [==============================] - 0s 160us/step - loss: 0.4683 - acc: 0.7868
Epoch 334/1000
999/999 [==============================] - 0s 156us/step - loss: 0.4669 - acc: 0.7808
Epoch

...

999/999 [==============================] - 0s 145us/step - loss: 0.4568 - acc: 0.7838
Epoch 650/1000
999/999 [==============================] - 0s 142us/step - loss: 0.4814 - acc: 0.7768
Epoch 651/1000
999/999 [==============================] - 0s 148us/step - loss: 0.4800 - acc: 0.7718
Epoch 652/1000
999/999 [==============================] - 0s 146us/step - loss: 0.4638 - acc: 0.7848
Epoch 653/1000
999/999 [==============================] - 0s 152us/step - loss: 0.4627 - acc: 0.7788
Epoch 654/1000
999/999 [==============================] - 0s 136us/step - loss: 0.4689 - acc: 0.7738
Epoch 655/1000
999/999 [==============================] - 0s 142us/step - loss: 0.4686 - acc: 0.7808
Epoch 656/1000
999/999 [==============================] - 0s 147us/step - loss: 0.4608 - acc: 0.7778
Epoch 657/1000
999/999 [==============================] - 0s 138us/step - loss: 0.4574 - acc: 0.7928
Epoch 658/1000
999/999 [==============================] - 0s 136us/step - loss: 0.4567 - acc: 0.7918
Epoch

Epoch 730/1000
999/999 [==============================] - 0s 164us/step - loss: 0.4576 - acc: 0.7838
Epoch 731/1000
999/999 [==============================] - 0s 166us/step - loss: 0.4658 - acc: 0.7868
Epoch 732/1000
999/999 [==============================] - 0s 153us/step - loss: 0.5001 - acc: 0.7608
Epoch 733/1000
999/999 [==============================] - 0s 142us/step - loss: 0.4847 - acc: 0.7738
Epoch 734/1000
999/999 [==============================] - 0s 148us/step - loss: 0.4545 - acc: 0.7908
Epoch 735/1000
999/999 [==============================] - 0s 145us/step - loss: 0.4575 - acc: 0.7878
Epoch 736/1000
999/999 [==============================] - 0s 144us/step - loss: 0.4752 - acc: 0.7848
Epoch 737/1000
999/999 [==============================] - 0s 151us/step - loss: 0.4695 - acc: 0.7758
Epoch 738/1000
999/999 [==============================] - 0s 148us/step - loss: 0.4672 - acc: 0.7818
Epoch 739/1000
999/999 [==============================] - 0s 137us/step - loss: 0.4660 - ac

999/999 [==============================] - 0s 147us/step - loss: 0.4639 - acc: 0.7698
Epoch 812/1000
999/999 [==============================] - 0s 136us/step - loss: 0.4670 - acc: 0.7848
Epoch 813/1000
999/999 [==============================] - 0s 162us/step - loss: 0.4623 - acc: 0.7828
Epoch 814/1000
999/999 [==============================] - 0s 141us/step - loss: 0.4571 - acc: 0.7828
Epoch 815/1000
999/999 [==============================] - 0s 146us/step - loss: 0.4670 - acc: 0.7858
Epoch 816/1000
999/999 [==============================] - 0s 136us/step - loss: 0.4708 - acc: 0.7798
Epoch 817/1000
999/999 [==============================] - 0s 151us/step - loss: 0.4548 - acc: 0.7828
Epoch 818/1000
999/999 [==============================] - 0s 144us/step - loss: 0.4600 - acc: 0.7748
Epoch 819/1000
999/999 [==============================] - 0s 138us/step - loss: 0.4615 - acc: 0.7798
Epoch 820/1000
999/999 [==============================] - 0s 151us/step - loss: 0.4537 - acc: 0.7818
Epoch

999/999 [==============================] - 0s 147us/step - loss: 0.4580 - acc: 0.7858
Epoch 893/1000
999/999 [==============================] - 0s 145us/step - loss: 0.4585 - acc: 0.7818
Epoch 894/1000
999/999 [==============================] - 0s 137us/step - loss: 0.4593 - acc: 0.7908
Epoch 895/1000
999/999 [==============================] - 0s 133us/step - loss: 0.4564 - acc: 0.7918
Epoch 896/1000
999/999 [==============================] - 0s 157us/step - loss: 0.4534 - acc: 0.7878
Epoch 897/1000
999/999 [==============================] - 0s 150us/step - loss: 0.4596 - acc: 0.7808
Epoch 898/1000
999/999 [==============================] - 0s 148us/step - loss: 0.4556 - acc: 0.7838
Epoch 899/1000
999/999 [==============================] - 0s 151us/step - loss: 0.4584 - acc: 0.7868
Epoch 900/1000
999/999 [==============================] - 0s 139us/step - loss: 0.4524 - acc: 0.7948
Epoch 901/1000
999/999 [==============================] - 0s 145us/step - loss: 0.4518 - acc: 0.7878
Epoch

999/999 [==============================] - 0s 191us/step - loss: 0.4535 - acc: 0.8038
Epoch 973/1000
999/999 [==============================] - 0s 176us/step - loss: 0.4521 - acc: 0.7968
Epoch 974/1000
999/999 [==============================] - 0s 165us/step - loss: 0.4536 - acc: 0.7898
Epoch 975/1000
999/999 [==============================] - 0s 153us/step - loss: 0.4530 - acc: 0.7978
Epoch 976/1000
999/999 [==============================] - 0s 155us/step - loss: 0.4502 - acc: 0.7888
Epoch 977/1000
999/999 [==============================] - 0s 169us/step - loss: 0.4423 - acc: 0.7918
Epoch 978/1000
999/999 [==============================] - 0s 181us/step - loss: 0.4562 - acc: 0.7888
Epoch 979/1000
999/999 [==============================] - 0s 152us/step - loss: 0.4499 - acc: 0.7988
Epoch 980/1000
999/999 [==============================] - 0s 160us/step - loss: 0.4571 - acc: 0.7918
Epoch 981/1000
999/999 [==============================] - 0s 169us/step - loss: 0.4507 - acc: 0.7938
Epoch

In [17]:
class Chromosome : 
    def __init__(self,genNum,trueProb) :
        self.chrom = np.array([1 if random() > trueProb else 0 for _ in range(genNum) ])
    
    def mutate(prob) :
        pass

In [18]:
class Population :
    def __init__(self,popSize,genNum,trueProb) :
        self.pop = np.array([])
        for _ in range(popSize) :
            np.append(self.pop,Chromosome(genNum,trueProb))
        
    def mutate(self,prob) :
        pass
    
    def crossover(self,prob) :
        pass


In [20]:
class GeneticAlgorithm :
    def __init__(self,initPopSize,genNum,trueProb) :
        self.initPop = Population(initPopSize,genNum,trueProb)
        self.bestChrom = None
        self.bestFitness = float('-inf')
        
    def run(self,mProb,cProb) :
        pass